# Proyecto 03 - Series de Tiempo

## Dataset: Flujo Vehicular por Unidades de Peaje AUSA


### 1. Preparación del Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
data_2019 = pd.read_csv('flujo-vehicular-2019.csv', sep =',')
data_2018 = pd.read_csv('flujo-vehicular-2018.csv', sep =',')
data_2017 = pd.read_csv('flujo-vehicular-2017.csv', sep =',')

##### Podemos decir entonces que los años 2017, 2018 y 2019 tienen las siguientes columnas con la siguiente información:

**periodo**: año del dset 

**fecha**: año dia mes

**hora_inicio** hora de inicio de la cuenta de vehiculos (DROP)

**hora_fin** hora de fin de la cuenta de vehiculos (Vamos a usar esta para tomar la hora)

**dia**: dia de la semana 

**estacion**: estaciones de peaje, difieren levemente entre los años (Dellepiane se unifica en 2019, antes habia Dellepiane Liniers y Centro, y se agrega PDB)

**sentido**: Centro o Provincia

**tipo_vehiculo**: Liviano o Pesado

**forma_pago**: Hay varias y varian entre los años.

**cantidad_pasos**: Cantidad de vehiculos

In [ ]:
dataset = pd.concat([data_2019, data_2018, data_2017])
dataset.drop(columns = ['periodo','hora_inicio'], inplace = True)
dataset.head()

In [ ]:
dataset['dia'].value_counts()

- Observación: Existe una clara disminución de la cantidad de vehiculos que circulan en la autopista los fines de semana.

In [ ]:
dataset['fecha2'] = pd.to_datetime(dataset.fecha) + pd.to_timedelta(dataset.hora_fin, unit = 'h')
dataset = dataset.set_index('fecha2')
dataset.drop(columns=['fecha','hora_fin'], inplace = True)
dataset.head()

In [ ]:
dataset.reset_index()

In [ ]:
dataset.rename(index = {'fecha2':'fecha'}, inplace = True) 

In [ ]:
dataset['fecha']=dataset.index
dataset['hora'] = dataset['fecha'].dt.hour
dataset['dayofweek'] = dataset['fecha'].dt.dayofweek
dataset['trimestre'] = dataset['fecha'].dt.quarter
dataset['mes'] = dataset['fecha'].dt.month
dataset['año'] = dataset['fecha'].dt.year
dataset['dayofyear'] = dataset['fecha'].dt.dayofyear
dataset['dayofmonth'] = dataset['fecha'].dt.day
dataset['weekofyear'] = dataset['fecha'].dt.weekofyear

In [ ]:
holidays2017=['2017-01-01','2017-02-27','2017-02-28','2017-03-28','2017-04-02','2017-04-13','2017-04-14','2017-05-01','2017-06-17','2017-06-20','2017-07-09','2017-08-17','2017-10-12','2017-11-20','2017-12-08','2017-12-25']
holidays2018=['2018-01-01','2018-02-12','2018-02-13','2018-03-24','2018-03-30','2018-04-02','2018-04-30','2018-05-01','2018-05-25','2018-06-17','2018-06-20','2018-07-09','2018-08-20','2018-10-15','2018-11-19','2018-12-08','2018-12-24','2018-12-25','2018-12-31']
holidays2019=['2019-01-01','2019-03-04','2019-03-05','2019-03-24','2019-04-02','2019-04-18','2019-04-19','2019-05-01','2019-05-25','2019-06-17','2019-06-20','2019-07-08','2019-07-09','2019-08-17','2019-08-19','2019-10-12','2019-10-14','2019-11-18','2019-12-08','2019-12-25']

### PARTE A: Análisis Exploratorio de datos

##### A) 1. Comportamiento del flujo de automóviles por año

- Para realizar un análisis más detallado del comportamiento por año, utilizaré sólo la Estación **Illia** y el medio de pago **efectivo**.

In [ ]:
datasetillia=dataset[dataset['estacion']=='Illia']
datasetillia=datasetillia[datasetillia['forma_pago']=='EFECTIVO']
diarioillia = datasetillia.resample('D', on = 'fecha').sum()
diarioillia.drop(['hora','dayofweek','trimestre','mes','año','dayofyear','dayofmonth','weekofyear'],axis=1,inplace=True)
diarioillia.reset_index()

In [ ]:
aut_2017=diarioillia.loc['2017-01-01':'2017-12-31']
aut_2018=diarioillia.loc['2018-01-01':'2018-12-31']
aut_2019=diarioillia.loc['2019-01-01':'2019-12-31']

In [ ]:
print(f'Cantidad de registros por año:\n2017: {aut_2017.cantidad_pasos.sum()}\n2018: {aut_2018.cantidad_pasos.sum()}\n2019: {aut_2019.cantidad_pasos.sum()}')

- Observación: Existe una marcada tendencia a la disminución del pago en efectivo a lo largo del tiempo. Puede que sea por el reemplazo por la forma de pago "Telepase"

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
fig = px.line(aut_2017, x=aut_2017.index, y='cantidad_pasos',title='Cantidad de autos en 2017 para la autopista Illia')
fig.add_trace(go.Scatter(x=holidays2017, y=aut_2017.loc[holidays2017].cantidad_pasos,
                    mode='markers', name='Feriados'))
fig.update_layout(xaxis_title='Días 2017',
                   yaxis_title='Cantidad de automóviles')
fig.update_xaxes(nticks=100,showgrid=False)
fig.show()
fig2 = px.line(aut_2018, x=aut_2018.index, y='cantidad_pasos',title='Cantidad de autos en 2018 para la autopista Illia')
fig2.add_trace(go.Scatter(x=holidays2018, y=aut_2018.loc[holidays2018].cantidad_pasos,
                    mode='markers', name='Feriados'))
fig2.update_layout(xaxis_title='Días 2018',
                   yaxis_title='Cantidad de automóviles')
fig2.update_xaxes(nticks=100,showgrid=False)
fig2.show()
fig3 = px.line(aut_2019, x=aut_2019.index, y='cantidad_pasos',title='Cantidad de autos en 2019 para la autopista Illia')
fig3.add_trace(go.Scatter(x=holidays2019, y=aut_2019.loc[holidays2019].cantidad_pasos,
                    mode='markers', name='Feriados'))
fig3.update_layout(xaxis_title='Días 2019',
                   yaxis_title='Cantidad de automóviles')
fig3.update_xaxes(nticks=100,showgrid=False)
fig3.show()

- Los **puntos en rojo** indican los **feriados nacionales** que, como es de esperar, puede producir una disminución notable del flujo de automóviles.
- Sin embargo, existen otros picos "para abajo" que pueden deberse a paros nacionales o días Domingo/vacaciones.

##### A) 2. A qué se deben los outliers en el dataset?

###### 2017: Teniendo en cuenta lo dispuesto en el gráfico, se fija un umbral de 27.000 automóviles para ver qué ocurrió los días donde se vieron menor cantidad de registros

In [ ]:
out2017=aut_2017[aut_2017['cantidad_pasos']<27000]
print(out2017)

- El 6 de Abril de 2017 hubo un paro nacional.
- El 9 de Octubre de 2017 fue Domingo, puede ser un factor determinante para la disminución de automóviles en las autopistas.
- El 24 y 31 de Diciembre son días festivos y atípicos en cuanto a la cantidad de automóviles circulando en las autopistas.

###### 2018: Teniendo en cuenta lo dispuesto en el gráfico, se fija un umbral de 25.000 automóviles para ver qué ocurrió los días donde se vieron menor cantidad de registros

In [ ]:
out2018=aut_2018[aut_2018['cantidad_pasos']<25000]
print(out2018)

- El 25 de Junio de 2018 hubo paro nacional.
- El 25 de Septiembre de 2018 hubo paro nacional.
- El 29 de Noviembre fue el comienzo del G20 en Argentina.
- El 30 de Noviembre y el 1 de diciembre se decreta feriado en CABA por la celebración de la Cumbre de Líderes del Grupo de los 20 en la ciudad.
- El 24, 30 y 31 de Diciembre son días festivos y atípicos en cuanto a la cantidad de automóviles circulando en las autopistas.

###### 2019: Teniendo en cuenta lo dispuesto en el gráfico, se fija un umbral de 22.000 automóviles para ver qué ocurrió los días donde se vieron menor cantidad de registros

In [ ]:
out2019=aut_2019[aut_2019['cantidad_pasos']<22000]
print(out2019)

- No existió ningún hecho relevante que explique la disminución en la cantidad de personas que se registró el 13 de Enero. Una posible razón podría ser las vacaciones de verano, sumado a que fue Domingo.
- El 29 de Mayo hubo paro nacional.
- El 17 de Junio fue feriado Nacional, conmemorando la muerte del General Martín Miguel de Guemes.
- No existió ningún hecho relevante que explique la disminución en la cantidad de personas que se registró el 1 de Diciembre. Una posible razón podría ser que fue Domingo.
- El  31 de Diciembre es día festivo y atípico en cuanto a la cantidad de automóviles circulando en las autopistas

- Si bien existen días en 0, no imputaré su valor ya que es sólo para la forma de pago "efectivo". Al tomar la totalidad de las formas de pago y de estaciones, se puede notar que existen datos para las otras opciones, por lo que no me parece necesario realizar ninguna imputación.

##### A) 3. Detección de horas pico por día y por sentido

In [ ]:
lunes=dataset[dataset['dia']=='Lunes']
martes=dataset[dataset['dia']=='Martes']
mierc=dataset[dataset['dia']=='Miércoles']
jueves=dataset[dataset['dia']=='Jueves']
viernes=dataset[dataset['dia']=='Viernes']
sabado=dataset[dataset['dia']=='Sábado']
dom=dataset[dataset['dia']=='Domingo']

In [ ]:
phlunes=lunes.groupby(['hora'])['cantidad_pasos'].sum()
phmart=martes.groupby(['hora'])['cantidad_pasos'].sum()
phmierc=mierc.groupby(['hora'])['cantidad_pasos'].sum()
phjuev=jueves.groupby(['hora'])['cantidad_pasos'].sum()
phvier=viernes.groupby(['hora'])['cantidad_pasos'].sum()
phsab=sabado.groupby(['hora'])['cantidad_pasos'].sum()
phdom=dom.groupby(['hora'])['cantidad_pasos'].sum()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=phlunes,
                    mode='lines',
                    name='lunes'))
fig.add_trace(go.Scatter(y=phmart,
                    mode='lines',
                    name='martes'))
fig.add_trace(go.Scatter(y=phmierc,
                    mode='lines',
                    name='miercoles'))
fig.add_trace(go.Scatter(y=phjuev,
                    mode='lines',
                    name='jueves'))
fig.add_trace(go.Scatter(y=phvier,
                    mode='lines',
                    name='viernes'))
fig.add_trace(go.Scatter(y=phsab,
                    mode='lines',
                    name='sabado'))
fig.add_trace(go.Scatter(y=phdom,
                    mode='lines',
                    name='domingo'))
fig.update_layout(title='Cantidad de automóviles que cruzan el telepeaje por día y por hora (total 2017,2018 y 2019)',
                   xaxis_title='Hora del día',
                   yaxis_title='Cantidad de automóviles')
fig.update_xaxes(nticks=24,showgrid=False)
fig.show()

#### Conclusiones: 
    - El viernes es el día con mayor cantidad de tránsito en autopistas.
    - Considerando únicamente los días laborales, se puede concluir que existen horas pico en las autopistas, independientemente del día laboral que se tenga en cuenta:
            *Por la mañana, las horas picos son a las 8 y 9 hs.
            *Durante la mitad de la tarde, los picos máximos se dan a las 13 y 16 hs.
            *En la "vuelta a casa/salida del trabajo" se dan otros dos picos de circulación en autopista: 18 y 19 hs.
    - Durante días Sábados y Domingos disminuye notablemente la cantidad de automóviles circulando, aunque también se vuelve a alcanzar un pico a las 13/14 hs y a las 19 hs durante el fin de semana.


In [ ]:
direc=dataset.groupby(['hora','sentido'])[('cantidad_pasos')].sum().reset_index(name='cantidad_pasos')
direc=pd.DataFrame(direc)

In [ ]:
import seaborn as sns
plt.figure(figsize=(18,6))
sns.barplot(data=direc, x='hora',y='cantidad_pasos',hue='sentido', palette='RdPu')
plt.xlabel('Hora')
plt.ylabel('Cantidad de automóviles')
plt.title('Flujo de automóviles clasificados por sentido según horario')

#### Conclusiones: 
    - Hay una marcada diferencia entre la cantidad de autos que se desplazan hacia provincia y los que ingresan a Capital Federal.
    - El gráfico de barras permite diferenciar que, al comienzo de la jornada laboral, existen mayor cantidad de automóviles que se desplazan de Provincia hacia el centro de Capital Federal. Al terminarla misma, la tendencia vuelve a cambiar.

##### A) 3. Análisis de la base de datos por estación

In [ ]:
estaciones=dataset['estacion'].unique()
print(f'Existen {len(estaciones)} estaciones en el dataset, cuyos nombres son: {estaciones}')

In [ ]:
fig = px.pie(dataset, values='cantidad_pasos', names='estacion', title='Cantidad de registros por estación')
fig.show()

#### Conclusiones: 
 Las dos estaciones con mayor cantidad de registros son **Illia** y **Avellaneda**.

##### A) 4. Análisis de la forma de pago por año

In [ ]:
form_2017=dataset[dataset['año']==2017]
form_2018=dataset[dataset['año']==2018]
form_2019=dataset[dataset['año']==2019]

In [ ]:
form=dataset.groupby(['forma_pago','año'])['cantidad_pasos'].sum().reset_index(name='cantidad_pasos')
fig = px.bar(form, x="año", y="cantidad_pasos", color="forma_pago", title="Cambios en la forma de pago por año")
fig.show()

#### Conclusión: 
    - A medida que transcurren los años se evidencia un reemplazo del efectivo por el telepase, lo cual era de esperar debido a la implementación de este nuevo medio de pago también fue acompañado por polìticas de reducción de tarifas para los usuarios.


### PARTE B: Modelo de Machine Learning

**Pasos a seguir en la Parte B:**
       - 1: Evaluar el comportamiento de la serie y sus componentes.
       - 2: Evaluar estacionariedad para poder predecir.
       - 3: Establecer un modelo Benchmark para poder comparar resultados.
       - 4: Predicción del último trimestre 2019 con Prophet.
       - 5: Predicción del último trimestre 2019 con XGBoost.

**Métricas a utilizar**:
    - RMSE: Raíz del error medio cuadrático.

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import pmdarima as pm
from pmdarima.model_selection import SlidingWindowForecastCV
from pmdarima.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

- Para el modelo de predicción, utilizaré la información disponible de las estaciones **Illia** y **Avellaneda**.

In [ ]:
dataset = dataset[dataset['estacion'].str.contains('Illia|Avellaneda')]
dataset.head(3)

In [ ]:
dataset['fecha']=dataset.index
diario = dataset.resample('D', on = 'fecha').sum()

In [ ]:
diario['fecha']=diario.index
diario['hora'] = diario['fecha'].dt.hour
diario['dayofweek'] = diario['fecha'].dt.dayofweek
diario['trimestre'] = diario['fecha'].dt.quarter
diario['mes'] = diario['fecha'].dt.month
diario['año'] = diario['fecha'].dt.year
diario['dayofyear'] = diario['fecha'].dt.dayofyear
diario['dayofmonth'] = diario['fecha'].dt.day
diario['weekofyear'] = diario['fecha'].dt.weekofyear

In [ ]:
diario.drop(['fecha','hora','dayofweek','trimestre','año','dayofyear','dayofmonth','weekofyear'],axis=1,inplace=True)

In [ ]:
print(f'Promedio de autos por día en las dos autopistas utilizadas: {round(diario.cantidad_pasos.mean(),3)}')

##### B) 1. Evaluación del comportamiento de la serie diaria

In [ ]:
import plotly.express as px
fig = go.Figure()
fig.add_trace(go.Scatter(x=diario.index, y=diario['cantidad_pasos'], mode='lines'))
fig.update_layout(title='Cantidad de autos por día (2017-2019)',
                   xaxis_title='Día',
                   yaxis_title='Cantidad autos')

##### B) 1. Evaluación del comportamiento de la serie mensual

In [ ]:
mensual = dataset.resample('M', on = 'fecha').sum()
fig = go.Figure()
fig.add_trace(go.Scatter(x=mensual.index, y=mensual['cantidad_pasos'], mode='lines+markers'))
fig.update_layout(title='Cantidad de autos por mes (2017-2019)',
                   xaxis_title='Mes',
                   yaxis_title='Cantidad autos')

In [ ]:
diariograf=diario.loc['2019-03-01':'2019-03-31']
fig = go.Figure()
fig.add_trace(go.Scatter(x=diariograf.index, y=diariograf['cantidad_pasos'], mode='lines+markers'))
fig.update_layout(title='Cantidad de autos en un mes (Marzo 2019)',
                   xaxis_title='Mes',
                   yaxis_title='Cantidad autos')

- Existe una marcada tendencia a que **disminuya la cantidad de automóviles en Febrero**. El comportamiento es idéntico en los tres años de análisis

- Otra forma de ver el comportamiento de la serie es a través de la decomposición de su tendencia, estacionalidad y residuos

In [ ]:
result = seasonal_decompose(diario.cantidad_pasos, model='additive')
plt.rcParams['figure.figsize'] = [12,12]
result.plot()
plt.show()

* Otra forma de ver la tendencia

In [ ]:
import statsmodels.api as sm
ciclo, tend = sm.tsa.filters.hpfilter(diario['cantidad_pasos'])
diario['tend'] = tend
diario[['cantidad_pasos', 'tend']].plot(figsize=(18, 6), fontsize=12)

#### Conclusiones de 1:
    - La serie presenta una estacionalidad anual, que se observa al ver una marcada reducción de la cantidad de usuarios hacia enero/febrero y también semanal, que puede verse en detalle al disminuir la frecuencia de datos y observar que para los fines de semana la cantidad de usuarios baja notablemente.

##### B) 2. Evaluación de la estacionariedad de la serie 

- Es necesario trabajar con una serie estacionaria, ya que las predicciones suelen ser más confiables. Es por esto que se va a chequear la estacionariedad de la serie a través del test de Dickey-Fuller, donde la hipótesis nula es que la serie no es estacionaria. Por lo tanto, el *p-value* tiene que permitir rechazar H0.

In [ ]:
def adf_test(series, title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print('Test de Dickey Fuller: {}'.format(title))
    # .dropna() handles differenced data
    result = adfuller(series.dropna(),autolag='AIC') 
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out['critical value ({})'.format(key)]=val
        
    # .to_string() removes the line "dtype: float64"
    print(out.to_string())          
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
    else:
        print("Weak evidence against the null hypothesis")
        
        
adf_test(diario['cantidad_pasos'],title='') 

#### Conclusión:
    -Con un p-value menor a 0.01, existe evidencia suficiente para confirmar que la serie "cantidad_pasos" es estacionaria (no es una serie random walk).

##### Autocorrelación de la serie

In [ ]:
plot_acf(diario['cantidad_pasos'], lags=90)

- Existe autocorrelación de su variable con su rezago cada 7 días, lo cual tiene sentido ya que la cantidad de automóviles que circulan depende del día del que se tenga en consideración.

- La presencia de autocorrelación permite confirmar que los rezagos de la variable pueden usarse para predecir su valor a futuro.

##### B) 3. Establecer un Modelo Benchmark

Observación a tener en cuenta para comparar RMSE:

In [ ]:
print(f'Cantidad promedio diario de automóviles que transitan por Illia y Avellaneda: {round(diario.cantidad_pasos.mean(),3)}')

In [ ]:
diario['shift_7']=diario.cantidad_pasos.shift(periods=7)

In [ ]:
split_date = '2019-10-01'
test = diario.loc[diario.index >= split_date].copy()

- RMSE del Modelo benchmark:

In [ ]:
ybench=test.cantidad_pasos
ypredbench=diario.loc['2019-10-01':'2019-12-31'].shift_7

In [ ]:
rmsebench = np.sqrt(mean_squared_error(ybench, ypredbench))
print(f'RMSE del Modelo Benchmark: {np.round(rmsebench,3)}')

##### B) 4. a) Predicción del último trimestre 2019 con Prophet.

In [ ]:
from fbprophet import Prophet

* Train/test split utilizando como corte el último trimestre de 2019

In [ ]:
diario.rename(columns={'fecha':'ds',
                     'cantidad_pasos':'y'},inplace=True)

In [ ]:
diario['ds']=diario.index
train = diario.loc[diario.index < split_date].copy()
test = diario.loc[diario.index >= split_date].copy()

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(train['y'],label='Data en train',color='green')
plt.plot(test['y'],label='Data en test',color='blue')
plt.legend()
plt.xlabel('Fecha')
plt.ylabel('Cantidad de autos')
plt.show()

* Predicción de la cantidad de automóviles para el último trimestre de 2019 utilizando Prophet

In [ ]:
prop = Prophet(weekly_seasonality=True)
prop.fit(train)
predic = prop.make_future_dataframe(periods=92, freq='D')
forecast = prop.predict(predic)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
\f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = prop.plot(forecast,
                 ax=ax)
plt.show()

In [ ]:
fig2 = prop.plot_components(forecast)

- Los componentes de la predicción parecen mostrar una igual a la esperada: la tendencia muestra una baja hacia 2019, lo cual se condice con los datos analizados, como asi lo hace la frecuencia semanal, la cual disminuye los fines de semana. Lo mismo ocurre con la frecuencia anual, ya que se muestra un descenso hacia los meses de vacaciones, como lo es Enero.

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = prop.plot(forecast,
                 ax=ax)
plt.scatter(test['ds'],test['y'], color='r')
plt.show()

In [ ]:
from fbprophet.plot import plot_plotly, plot_components_plotly
plot_plotly(prop, forecast)

* RMSE del modelo Prophet

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
rmseprop = (mean_squared_error(test['y'], forecast.yhat[-92:]))
print(f'RMSE: {round(np.sqrt(rmseprop),3)}')

- El modelo mejora su RMSE con respecto al benchmark establecido (de 53.686 pasa a 39.393). No obstante, intentaré mejorar la predicción con Prophet utilizando Cross Validation.

##### B) 4. b) Cross Validation con Prophet.

In [ ]:
from fbprophet.diagnostics import cross_validation

In [ ]:
cvprop= cross_validation(prop, initial='366 days',horizon='92 days')

In [ ]:
from fbprophet.diagnostics import performance_metrics
metric = performance_metrics(cvprop)
metric

In [ ]:
print(f'RMSE para un horizonte de 92 días: {round(metric.iloc[82,2],3)}')

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(cvprop, metric='rmse')

- El gráfico del RMSE muestra que, si bien el modelo presenta ciertas mediciones muy erráticas (outliers) antes de los 20 días de horizonte y después de los 60, en general se mantiene en un rmse esperado para el resto de las observaciones

- Haciendo Cross Validation teniendo en cuenta el mismo horizonte temporal (92 días, haciendo referencia al último trimestre del 2019) se achica el RMSE del modelo, pasando de 39.323 a 37.766.

#### Conclusiones de hacer forecast con Prophet:
    - El RMSE no resulta muy significativo, teniendo en cuenta que el promedio de autos que pasan por día sumando las dos autopistas (Illia y Avellaneda) es de 229.208. 
    - Lo bueno del modelo es que tiene en cuenta perfectamente la estacionalidad de la serie (semanal), haciendo la predicción mucho más exacta.

##### B) 5. a) Predicción del último trimestre de 2019 con XGBoost.

In [ ]:
import xgboost as xgb
from xgboost import plot_importance, plot_tree

In [ ]:
dataset.drop(['dia', 'estacion', 'sentido', 'tipo_vehiculo', 'forma_pago'], axis=1, inplace = True)

In [ ]:
dia = dataset.resample('D', on = 'fecha').sum()

In [ ]:
dia['fecha']=dia.index
dia['dayofweek'] = dia['fecha'].dt.dayofweek
dia['trimestre'] = dia['fecha'].dt.quarter
dia['mes'] = dia['fecha'].dt.month
dia['año'] = dia['fecha'].dt.year
dia['dayofyear'] = dia['fecha'].dt.dayofyear
dia['dayofmonth'] = dia['fecha'].dt.day
dia['weekofyear'] = dia['fecha'].dt.weekofyear

In [ ]:
dia.drop(['fecha'],axis=1,inplace=True)

* Train/test split utilizando como corte el último trimestre de 2019

In [ ]:
split_date = '10-01-2019'
xg_train = dia.loc[dia.index < split_date].copy()
xg_test = dia.loc[dia.index >= split_date].copy()

In [ ]:
X_train=xg_train.drop(['cantidad_pasos'],axis=1)
y_train=xg_train['cantidad_pasos']
X_test=xg_test.drop(['cantidad_pasos'],axis=1)
y_test=xg_test['cantidad_pasos']

In [ ]:
reg = xgb.XGBRegressor(n_estimators=100)
reg.fit(X_train, y_train)

* Predicción de la cantidad de automóviles para el último trimestre de 2019 utilizando XGBoost

In [ ]:
y_test_predxg = reg.predict(X_test)
y_train_predxg = reg.predict(X_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
rmse_trainxg = np.sqrt(mean_squared_error(y_train, y_train_predxg))
rmse_testxg = np.sqrt(mean_squared_error(y_test, y_test_predxg))
print(f'Raíz del error cuadrático medio en Train: {np.round(rmse_trainxg,5)}')
print(f'Raíz del error cuadrático medio en Test: {np.round(rmse_testxg,5)}')
plt.figure(figsize = (14,4))

plt.subplot(1,2,1)
sns.distplot(y_train - y_train_predxg, bins = 20, label = 'train',color='yellow')
sns.distplot(y_test - y_test_predxg, bins = 20, label = 'test',color='green')
plt.xlabel('errores')
plt.legend()


ax = plt.subplot(1,2,2)
ax.scatter(y_test,y_test_predxg, s =2,marker='*',c='purple')

lims = [
np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes]
]

ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
plt.xlabel('y (test)')
plt.ylabel('y_pred (test)')

plt.tight_layout()
plt.show()

In [ ]:
imp = plot_importance(reg, height=0.6)

* RMSE del modelo XGBoost

In [ ]:
rmsexg = np.sqrt(mean_squared_error(y_test, y_test_predxg))
print(f'RMSE de modelo XGBoost: {round((rmsexg),3)}')

- El modelo de predicción con XGBoost mejora su RMSE con respecto al modelo benchmark establecido (de 53.686 pasa a 36.367). No obstante, intentaré mejorar su performance utilizando Cross Validation

##### B) 5. b) Cross Validation con XGBoost.

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'gamma':np.arange(0,10,1),'eta':np.arange(0,1,0.1), 'n_estimators': np.arange(100,150,30)}

xg_regopt = GridSearchCV(reg, parameters, verbose=True)

xg_regopt.fit(X_train, y_train)

In [ ]:
print(f'Los parámetros óptimos son: {xg_regopt.best_params_}')

In [ ]:
y_test_optxg = xg_regopt.predict(X_test)
y_train_optxg = xg_regopt.predict(X_train)

- RMSE con XGBoost optimizado

In [ ]:
rmsexgopt = np.sqrt(mean_squared_error(y_test, y_test_optxg))
print(f'RMSE de modelo XGBoost: {round((rmsexgopt),3)}')

In [ ]:
rmse_trainxg = np.sqrt(mean_squared_error(y_train, y_train_optxg))
rmse_testxg = np.sqrt(mean_squared_error(y_test, y_test_optxg))
print(f'Raíz del error cuadrático medio en Train: {np.round(rmse_trainxg,5)}')
print(f'Raíz del error cuadrático medio en Test: {np.round(rmse_testxg,5)}')
plt.figure(figsize = (14,4))

plt.subplot(1,2,1)
sns.distplot(y_train - y_train_optxg, bins = 20, label = 'train',color='yellow')
sns.distplot(y_test - y_test_optxg, bins = 20, label = 'test',color='green')
plt.xlabel('errores')
plt.legend()


ax = plt.subplot(1,2,2)
ax.scatter(y_test,y_test_optxg, s =2,marker='*',c='purple')

lims = [
np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes]
]

ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
plt.xlabel('y (test)')
plt.ylabel('y_pred (test)')

plt.tight_layout()
plt.show()

#### Conclusiones de hacer predicción con XGBoost

- La predicción con XGBoost me permite alcanzar un RMSE más bajo que con Prophet (23.043), sin mejorar optimizando hiperparámetros con Cross Validation. Los hiperparámetros establecidos a evaluar son los que consideré adecuados según el modelo.

- A su vez, el análisis de la importancia relativa entre las variables que participan en la estimación permite observar que la hora es importante a la predicción, lo cual tiene sentido, y también el día de la semana a analizar, lo que puede dar indicio que estaría siendo necesario una dummy que capte los feriados o paros nacionales.

### PARTE C: Investigación

Existen limitaciones en el análisis de la PARTE B, que probablemente puedan ser eliminadas a partir de un análisis más detallado del dataset: 
    - En Argentina están los días feriados, que pueden ser detectados a través de la libreria datetime, pero también existen feriados "puente", que se crean mediante la Ley, o feriados trasladables, que si bien figuran como feriados pueden ser considerados días hábiles.
    - A su vez, el tráfico en Buenos Aires puede verse afectado también por paros nacionales, donde se reduce notablemente la cantidad de autos circulando, afectando a la predicción realizada. 

Es por esto que resulta fundamental **agregar estos datos para mejorar la predicción a través de variables booleanas.** El objetivo de la PARTE C será añadir estos datos para ver si mejora el RMSE del modelo.

In [ ]:
dataset = pd.concat([data_2019, data_2018, data_2017])
dataset.drop(columns = ['periodo','hora_inicio'], inplace = True)
dataset['fecha2'] = pd.to_datetime(dataset.fecha) + pd.to_timedelta(dataset.hora_fin, unit = 'h')
dataset = dataset.set_index('fecha2')
dataset.drop(columns=['fecha','hora_fin'], inplace = True)
dataset = dataset[dataset['estacion'].str.contains('Illia|Avellaneda')]

In [ ]:
dataset.reset_index()
dataset.rename({'fecha2':'fecha'}, inplace = True) 

In [ ]:
dataset['fecha']=dataset.index
dataset=dataset.resample('D', on='fecha').sum()

In [ ]:
dataset['fecha']=dataset.index
dataset['dayofweek'] = dataset['fecha'].dt.dayofweek
dataset['trimestre'] = dataset['fecha'].dt.quarter
dataset['mes'] = dataset['fecha'].dt.month
dataset['año'] = dataset['fecha'].dt.year
dataset['dayofyear'] = dataset['fecha'].dt.dayofyear
dataset['dayofmonth'] = dataset['fecha'].dt.day
dataset['weekofyear'] = dataset['fecha'].dt.weekofyear

In [ ]:
dataset.drop(['fecha'],axis=1,inplace=True)

##### C) 1. Feriados en Argentina

In [ ]:
import holidays

feriados= holidays.Argentina()
fer_2017=feriados['2017-01-01': '2017-12-31']
fer_2018=feriados['2018-01-01': '2018-12-31']
fer_2019=feriados['2019-01-01': '2019-12-31']

In [ ]:
feriados=[]
for i in sorted(fer_2017):
    feriados.append(i.strftime('%Y/%m/%d'))
for i in sorted(fer_2018):
    feriados.append(i.strftime('%Y/%m/%d'))
for i in sorted(fer_2019):
    feriados.append(i.strftime('%Y/%m/%d'))

##### C) 2. Paros nacionales en Argentina

In [ ]:
paros=['2017-04-06','2018-06-18','2018-09-25','2018-11-30','2019-05-29']
paros=pd.to_datetime(paros,format='%Y/%m/%d')
paros=pd.DataFrame(paros,columns=['paros'])

- El 30 de noviembre fue el día del G20 en Argentina, pero se puede tomar como un paro ya que disminuyó la cantidad de vehículos en circulación por una limitación del tránsito por parte del gobierno nacional.

In [ ]:
feriados_df = pd.DataFrame({'feriados': pd.to_datetime(feriados)})

In [ ]:
dataset = pd.merge(dataset, feriados_df, how = 'left',left_on='fecha',right_on = 'feriados').set_index(dataset.index)
dataset.loc[dataset.feriados.isna(), 'is_feriado'] = 0
dataset.loc[~dataset.feriados.isna(), 'is_feriado'] = 1

In [ ]:
dataset = pd.merge(dataset, paros, how = 'left',left_on='fecha',right_on = 'paros').set_index(dataset.index)
dataset.loc[dataset.paros.isna(), 'eventoext'] = 0
dataset.loc[~dataset.paros.isna(), 'eventoext'] = 1

In [ ]:
dataset.drop(['feriados'],axis=1,inplace=True)
dataset.drop(['paros'],axis=1,inplace=True)

In [ ]:
#Controlando la correcta señalización de feriados
print(f'Longitud del dataset de feriados: {len(feriados)}')
print(f'Nueva variable booleana para indicar feriados: {(dataset.is_feriado==1).sum()}')

In [ ]:
#Controlando la correcta señalización de paros
print(f'Longitud del dataset de feriados: {len(paros)}')
print(f'Nueva variable booleana para indicar feriados: {(dataset.eventoext==1).sum()}')

- Teniendo la nueva variable detallada para captar los posibles "outliers", haré nuevamente la predicción con XGBoost para ver si mejora el RMSE.

In [ ]:
split_date = '10-01-2019'
xg_trainout = dataset.loc[dataset.index < split_date].copy()
xg_testout = dataset.loc[dataset.index >= split_date].copy()

In [ ]:
X_train=xg_trainout.drop(['cantidad_pasos'],axis=1)
y_train=xg_trainout['cantidad_pasos']
X_test=xg_testout.drop(['cantidad_pasos'],axis=1)
y_test=xg_testout['cantidad_pasos']

In [ ]:
regout = xgb.XGBRegressor(n_estimators=100)
regout.fit(X_train, y_train)

In [ ]:
y_test_out = regout.predict(X_test)
y_train_out = regout.predict(X_train)

In [ ]:
rmseout = np.sqrt(mean_squared_error(y_test, y_test_out))
print(f'RMSE de modelo XGBoost: {round((rmseout),3)}')

- El RMSE aumenta añadiendo las dos variables booleanas que tienen en cuenta los posibles outliers que pueden surgir los días feriados o los paros nacionales con respectoa  la circulación de vehículos por la autopista.

### Conclusiones a tener en cuenta para futuros trabajos con el dataset

- La posibilidad de acceder al medio de pago utilizado por cada vehículo permite poder medir en términos de eficiencia el cambio de efectivo hacia telepase, dos de los medios más utilizados por los usuarios.

- A su vez, sin tener en cuenta el contexto actual, otro trabajo que me gustaría profundizar en un futuro sería proyectar hacia qué año desaparecería cualquier medio de pago que no sea el telepase, para eficientizar el tiempo de espera en las cabinas de telepeaje.